In [ ]:
import scipy.linalg as LA
from matplotlib import animation
from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def H1diag(t):
    k = 2*np.pi*np.fft.fftfreq(npoints,d=dx)
    Es = k*k/2
    return Es

def H2diag(t,psi):
    return g/2*np.conjugate(psi)*psi

In [ ]:
def TimeStap(psi,t):
    psi  = np.exp(-1j*(dt/2)*H2diag(t,psi))*psi
    t   += dt/2
    psi_ = np.fft.fft(psi)
    psi_ = np.exp(-1j*dt*H1diag(t))*psi_
    t   += dt/2
    psi  = np.fft.ifft(psi_)
    psi  = np.exp(-1j*(dt/2)*H2diag(t,psi))*psi
    return psi

In [ ]:
def make_grid(npoints,L):
    dx=L/npoints
    return np.arange(-L/2,L/2,dx,complex)

def grey_soliton(grid,nu=0.5,z0=0):
    gamma=1/np.sqrt(1-nu**2)
    return 1j*nu+1/gamma*np.tanh((grid-z0)/(np.sqrt(2/g)*gamma))

def dark_soliton(grid,z0=0):
    return grey_soliton(grid,nu=0,z0=z0)

In [ ]:
g=0.01
eps = 1/np.sqrt(g)
L = 40*eps
npoints = 200
dx=L/npoints
dt = 10
tsteps = 500
grid  = make_grid(npoints,L)
psi0  = grey_soliton(grid, 0.5,-10*eps)
psi0 *= grey_soliton(grid,-0.5, 10*eps)

In [ ]:
def animateTimeEvolution(xrange=[-L/2/eps,L/2/eps],yrange=[0,1.1]):
    fig, ax = plt.subplots()
    # add more axis specifications here, e.g. axis limits
    line, = ax.plot(xrange,yrange)
    global psi_t
    psi_t = psi0

    def animate(t):
        # assign current wave function to y at time t, x is the vector of grid points (to be defined!)
        #y = TimeEvolution(psi0, t)
        global psi_t
        y = psi_t
        psi_t = TimeStap(psi_t,t)
        line.set_data((grid).real/eps, (np.conjugate(y)*y).real)
        plt.close()
        return (line,)

    anim = animation.FuncAnimation(fig, animate,
                                   frames=np.arange(0,tsteps*dt,dt), # t-values
                                   interval=50, # wait time before displaying new frame in ms
                                   blit=True)

    return anim.to_jshtml()

In [ ]:
HTML(animateTimeEvolution())

In [ ]:
plt.plot(np.real(grid)/eps, np.real(psi0))
plt.plot(np.real(grid)/eps, np.imag(psi0))